In [ ]:
%pip install nbformat
%pip install imbalanced-learn
%pip install lightgbm

In [ ]:
import nbformat

nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [ ]:
%run ml_project_normalized.ipynb

In [ ]:
import numpy as np
np.float = float  # Patch for libraries using deprecated np.float


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score, train_test_split
from sklearn.utils.class_weight import compute_class_weight
from lightgbm import LGBMClassifier
from scipy.stats import randint, uniform
from sklearn.metrics import accuracy_score, balanced_accuracy_score

import numpy as np
import pandas as pd

def train_classifier_lightgbm(
    X_train, y_train, X_test, rare_classes, cat_cols, custom_thresholds=None
):
    """
    Trains a LightGBM model using hyperparameter tuning.
    Applies custom thresholding during prediction if specified and compares standard vs. custom accuracy.

    Parameters:
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series or np.array): Training target values.
        X_test (pd.DataFrame): Test features.
        rare_classes (list): List of integer-encoded classes to be oversampled.
        cat_cols (set or list): Categorical columns to one-hot encode.
        custom_thresholds (dict, optional): Dict of class_label -> threshold.
            e.g., {0: 0.7, 1: 0.5, 2: 0.5} for class 0 = supermajority.

    Returns:
        best_estimator: Trained pipeline.
        test_predictions: Custom-thresholded predictions.
        df_importances: Feature importances from LightGBM.
    """
    # Compute balanced sample weights
    class_labels = np.unique(y_train)
    class_weights = compute_class_weight('balanced', classes=class_labels, y=y_train)
    class_weight_dict = dict(zip(class_labels, class_weights))
    sample_weights = np.array([class_weight_dict[y] for y in y_train])

    # Ensure categorical features are strings
    categorical_features = [col for col in X_train.columns if col in cat_cols]
    for col in categorical_features:
        X_train[col] = X_train[col].astype(str)
        X_test[col] = X_test[col].astype(str)

    # Define preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ("onehot", OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
        ],
        remainder='passthrough'
    )

    # Build pipeline with LightGBM classifier
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("lgbm", LGBMClassifier(objective='multiclass', verbosity=-1))
    ])

    # Hyperparameter search space for LGBM
    param_distributions = {
        "lgbm__num_leaves": randint(20, 150),
        "lgbm__learning_rate": uniform(0.01, 0.5),
        "lgbm__n_estimators": randint(50, 500),
    "lgbm__subsample":        uniform(0.3, 0.7),
    "lgbm__colsample_bytree": uniform(0.3, 0.7),
        "lgbm__min_child_samples": randint(1, 50),
        "lgbm__min_split_gain": uniform(0, 1.0),
        "lgbm__reg_alpha": uniform(0, 1.0),
        "lgbm__reg_lambda": uniform(0.5, 2.5)
    }

    # Stratified cross-validation setup
    stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Randomized search for best hyperparameters
    randomized_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_distributions,
        n_iter=5,
        cv=stratified_cv,
        scoring='balanced_accuracy',
        verbose=3,
        random_state=42
    )

    print(f"\n[INFO] Starting training with {len(X_train)} samples and {len(y_train)} labels")
    randomized_search.fit(X_train, y_train, **{'lgbm__sample_weight': sample_weights})
    print(f"[INFO] Training complete. Best model fitted on {len(X_train)} samples.\n")

    # Debug one hot encoded features by grabbing column names during training
    best_preprocessor = randomized_search.best_estimator_.named_steps['preprocessor']
    encoded_X_train = best_preprocessor.transform(X_train)
    if hasattr(encoded_X_train, "toarray"):
        encoded_X_train = encoded_X_train.toarray()
    feature_names = best_preprocessor.get_feature_names_out()
    print("Sample of transformed training features:")
    print(pd.DataFrame(encoded_X_train, columns=feature_names).head())

    # Print best parameters and CV score
    print('Best parameters:', randomized_search.best_params_)
    print('Best cross-validation balanced accuracy:', randomized_search.best_score_)

    # Generalization accuracy via cross_val_score
    cv_scores = cross_val_score(
        randomized_search.best_estimator_,
        X_train,
        y_train,
        cv=5,
        verbose=3,
        scoring='balanced_accuracy'
    )
    print('Generalization balanced accuracy:', cv_scores.mean())

    # Prediction with optional custom thresholds
    lgbm_model = randomized_search.best_estimator_.named_steps['lgbm']
    class_names = lgbm_model.classes_


    # Further helps with class imbalance by requiring certain labels achieve a higher/lower prediciton confidence
    def apply_custom_thresholds(probabilities, class_names, thresholds_dict):
        thresholds = np.array([thresholds_dict.get(cls, 0.5) for cls in class_names])
        preds = []
        for row in probabilities:
            passed = row >= thresholds
            if not passed.any():
                pred = class_names[np.argmax(row)]
            else:
                pred = class_names[np.argmax(passed * row)]
            preds.append(pred)
        return np.array(preds)

    # get probability distribution for all test records for all labels for custom thresholding
    proba_test = randomized_search.predict_proba(X_test)
    if custom_thresholds:
        print("[INFO] Applying custom threshold logic to test set")
        test_predictions = apply_custom_thresholds(proba_test, class_names, custom_thresholds)
    else:
        test_predictions = randomized_search.predict(X_test)

    # Compare accuracies on training set if custom thresholds provided
    if custom_thresholds:
        proba_train = randomized_search.predict_proba(X_train)
        preds_custom_train = apply_custom_thresholds(proba_train, class_names, custom_thresholds)
        acc_standard = accuracy_score(y_train, randomized_search.predict(X_train))
        acc_custom = accuracy_score(y_train, preds_custom_train)
        bal_acc_standard = balanced_accuracy_score(y_train, randomized_search.predict(X_train))
        bal_acc_custom = balanced_accuracy_score(y_train, preds_custom_train)

        print(f"\nAccuracy Comparison on Training Set:")
        print(f"Standard Accuracy: {acc_standard:.4f}")
        print(f"Custom Threshold Accuracy: {acc_custom:.4f}")
        print(f"Standard Balanced Accuracy: {bal_acc_standard:.4f}")
        print(f"Custom Threshold Balanced Accuracy: {bal_acc_custom:.4f}")

    # Extract feature importances
    importances = lgbm_model.feature_importances_
    df_importances = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    })

    return randomized_search.best_estimator_, test_predictions, df_importances


In [ ]:
# Near deadline attempt to add location into training. Little attempt before to try feature engineering on location.
def add_in_austin(df, df_loc):
    """
    Adds a binary column 'in_austin' that is 1 if 'austin' appears
    (case‑insensitive) in the 'location' field, else 0.
    """
    df['in_austin'] = (
        df_loc
        .str.contains('austin', case=False, na=False)
        .astype(int)
    )
    return df

In [ ]:

X_train = df_train.copy()
# X_train = add_in_austin(X_train, df_train_location) # df_train_location used to be populated in ml_project
X_train = bucket_seasons(X_train)
X_train = bucket_days(X_train)
X_train = X_train.drop(columns=['intake_month', 'intake_hour'])


In [ ]:

X_test = df_test.copy()
# X_test = add_in_austin(X_test, df_test_location)
X_test = bucket_seasons(X_test)
X_test = bucket_days(X_test)
X_test = X_test.drop(columns=['intake_month', 'intake_hour'])
X_test.head()


In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=DataConversionWarning)

# Encode the target variable.
le = LabelEncoder()
y_train = X_train['outcome_type']
X_train = X_train.drop(columns=['outcome_type'])
y_train = le.fit_transform(y_train)
print('Encoding mapping:', le.classes_)

# Identify rare classes that need oversampling.
rare_classes = [
    label for label, count in pd.Series(y_train).value_counts().items()
    if count < 0.05 * len(y_train)
]
print("Rare classes:")
for cls in rare_classes:
    print(f"  {cls}: {le.classes_[cls]}")

# Define column groups (this example will one-hot encode all columns, so cat_cols and num_cols are not used in the transformer).
cat_cols = {'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'intake_month', 'intake_hour', 'intake_year', 'season', 'time_of_day', 'size', 'primary_color'}
categorical_features = [col for col in X_train.columns if col in cat_cols]

'''
# cat_cols_freq = ['primary_color']


# Frequency encode selected high-cardinality features, old thing we tried before we realized this is data leakage
# for col in cat_cols_freq:
#   freq_map = X_train[col].value_counts()
#   X_train[col] = X_train[col].map(freq_map)
#   X_test[col]  = X_test[col].map(freq_map).fillna(0)
'''
# Train the classifier with the refactored pipeline.

custom_thresholds = {
    0: 0.6, # Adopted
    1: 0.5, # Died
    2: 0.5, # Euthanasia
    3: 0.5, # Return to Owner
    4: 0.5  # Transfer
}

best_model, test_predictions, df_importances = train_classifier_lightgbm(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    rare_classes=rare_classes,
    cat_cols=categorical_features,
    custom_thresholds=custom_thresholds
)

# Convert predictions back to original labels.
predictions = le.inverse_transform(test_predictions)

# Save predictions; assumes save_predictions is defined elsewhere.
classification_report_with_accuracy_score(y_test, predictions)
#save_predictions(predictions, 'lightgbm')

In [ ]:
import matplotlib.pyplot as plt

# plots feature importance values to see how our feature engineering is performing and what features we could possibly leave out
df_plot = df_importances.sort_values(by='importance', ascending=True)

plt.figure(figsize=(10, 12))
plt.barh(df_plot['feature'], df_plot['importance'])
plt.xlabel('Feature Importance')
plt.title('Feature Importances from light gbm')
plt.tight_layout()
plt.show()